In [1]:
from datetime import datetime # Date calculations
from dateutil.relativedelta import relativedelta # Exact date calcs
import pandas as pd #DataFrames
import numpy as np #
import math # Round up function
import random # Random loan term generation
from tqdm import tqdm # Loading bars

## Helper Functions

In [2]:
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return(math.ceil(n * multiplier) / multiplier)

# Return a random datetime between to datetime objects
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return(start + datetime.timedelta(seconds=random_second))

## Loan Terms

In [3]:
terms = {'principal':10000000,
 'cash_rate':0.05,
 'interest_reserve_rate':0.05,
 'interest_reserve_term':0.5,
 'origination_points':0.025,
 'origination_points_deferred':0.025,
 'date_originated':datetime.strptime('2019-01-01', '%Y-%m-%d'),
 'date_payoff':datetime.strptime('2020-01-01', '%Y-%m-%d')
}

terms_full_ir = {'principal':10000000,
 'cash_rate':0.05,
 'interest_reserve_rate':0.05,
 'interest_reserve_term':1,
 'origination_points':0.025,
 'origination_points_deferred':0.025,
 'date_originated':datetime.strptime('2019-01-01', '%Y-%m-%d'),
 'date_payoff':datetime.strptime('2020-01-01', '%Y-%m-%d')
}

def generate_loan_terms():
    principal = random.uniform(1000000,50000000)
    cash_rate = random.uniform(0.05,0.09)
    interest_reserve_rate = random.uniform(0, 0.05)
    interest_reserve_term = random.random()
    origination_points = random.uniform(0, 0.01)
    origination_points_deferred = random.uniform(0, 0.01)
    date_originated = random_date(datetime.strptime('2020-01-01', '%Y-%m-%d'), datetime.strptime('2022-01-01', '%Y-%m-%d'))
    date_payoff = date_originated + datetime.timedelta(int(random.uniform(12,24))*365/12)
    terms = {'principal':principal,'cash_rate':cash_rate,'interest_reserve_rate':interest_reserve_rate,'interest_reserve_term':interest_reserve_term,'origination_points':origination_points,'origination_points_deferred':origination_points_deferred,'date_originated':date_originated,'date_payoff':date_payoff,'periods':365,'frequency':'D'}
    return(terms)

## Loan CFs

In [4]:
def generate_loan(terms, amortizing = True, amortization_terms = {'term' : 30}):
    '''
    Periodic transactions.
    '''
    # Loan cash flows are calculated on a daily basis. Generate range of dates between origination and payoff date
    # # https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
    term = (terms['date_payoff'] - terms['date_originated']).days
    term_ir = round_up(term * terms['interest_reserve_term'],0)
    date_range = pd.date_range(terms['date_originated'], periods = term, freq = 'D')
    date_range_ir = pd.date_range(terms['date_originated'], periods = term_ir, freq = 'D')

    # Payments are calculated on a daily basis, on a 365/360 day convention, rounded to the nearest cent.
    rate_cash = terms['cash_rate'] / 360
    rate_ir = terms['interest_reserve_rate'] / 360

    rate_coupon = rate_cash + rate_ir
    n = ((terms['date_originated'] + relativedelta(years = amortization_terms['term'])) - terms['date_originated']).days

    if amortizing:
        coupon = round_up(((((1+rate_coupon) ** n) * rate_coupon) / (((1+rate_coupon) ** n) - 1)) * terms['principal'], 2)
    else:
        coupon = round_up(rate_coupon * terms['principal'], 2)
    
    balance_coupon = coupon * term
    balance_ir = balance_coupon * (rate_ir / rate_coupon) * terms['interest_reserve_term']
    print(balance_ir)
    cash = round_up(balance_coupon / term, 2)
    ir = round_up(balance_ir / term_ir, 2)

    daily_cash = pd.Series(cash, index = date_range)
    daily_ir = pd.Series(ir, index = date_range_ir)

    # Remove interest reserve from cash-flow stream
    daily_cash = daily_cash.subtract(daily_ir, fill_value = 0)


    '''
    One-time transactions
    '''
    rate = terms['cash_rate'] + (terms['interest_reserve_rate'] * terms['interest_reserve_term']) + terms['origination_points'] + terms['origination_points_deferred']
    points_in = terms['origination_points'] * terms['principal']
    points_out = terms['origination_points_deferred'] * terms['principal']
    if amortizing:
        delta = round_up(((((1+rate_coupon) ** n) * rate_coupon) / (((1+rate_coupon) ** n) - 1)) * terms['principal'], 2) - round_up(rate_coupon * terms['principal'], 2)
        delta = (delta * term) * (1 - balance_ir / balance_coupon)
        payoff = terms['principal'] + points_out - delta
    else:
        payoff = terms['principal'] + points_out
    net_funding = -(terms['principal'] - points_in - balance_ir)

    '''
    {
        'net_funding' : [{'2019-01-01' : -1000}],
        'cash' : [{'2019-01-01' : 1}]
    }
    '''

    # Create loan schedule

    net_funding = pd.Series(net_funding, index = pd.Index([terms['date_originated']]))
    payoff = pd.Series(payoff, index = pd.Index([terms['date_payoff']]))

    cash_flows = {
        'profit (loss)' : sum(net_funding) + sum(daily_cash) + sum(payoff),
        'net_funding' : net_funding,
        'cash' : daily_cash,
        'payoff' : payoff
    }

    return(cash_flows)

In [5]:
x = generate_loan(terms, amortizing = True)
x

266160.7375


{'profit (loss)': 1526576.2300000004, 'net_funding': 2019-01-01   -9.483839e+06
 dtype: float64, 'cash': 2019-01-01    1462.39
 2019-01-02    1462.39
 2019-01-03    1462.39
 2019-01-04    1462.39
 2019-01-05    1462.39
                ...   
 2019-12-27    2916.83
 2019-12-28    2916.83
 2019-12-29    2916.83
 2019-12-30    2916.83
 2019-12-31    2916.83
 Freq: D, Length: 365, dtype: float64, 'payoff': 2020-01-01    1.021194e+07
 dtype: float64}

In [6]:
y = generate_loan(terms, amortizing = False)
y

253472.42500000002


{'profit (loss)': 1513888.8250000048, 'net_funding': 2019-01-01   -9496527.575
 dtype: float64, 'cash': 2019-01-01    1392.68
 2019-01-02    1392.68
 2019-01-03    1392.68
 2019-01-04    1392.68
 2019-01-05    1392.68
                ...   
 2019-12-27    2777.78
 2019-12-28    2777.78
 2019-12-29    2777.78
 2019-12-30    2777.78
 2019-12-31    2777.78
 Freq: D, Length: 365, dtype: float64, 'payoff': 2020-01-01    10250000.0
 dtype: float64}

In [7]:
generate_loan(terms_full_ir, amortizing = True)

532321.475


{'profit (loss)': 1539264.4999999963, 'net_funding': 2019-01-01   -9217678.525
 dtype: float64, 'cash': 2019-01-01    1458.41
 2019-01-02    1458.41
 2019-01-03    1458.41
 2019-01-04    1458.41
 2019-01-05    1458.41
                ...   
 2019-12-27    1458.41
 2019-12-28    1458.41
 2019-12-29    1458.41
 2019-12-30    1458.41
 2019-12-31    1458.41
 Freq: D, Length: 365, dtype: float64, 'payoff': 2020-01-01    1.022462e+07
 dtype: float64}

In [8]:
generate_loan(terms_full_ir, amortizing = False)

506944.85000000003


{'profit (loss)': 1513889.700000003, 'net_funding': 2019-01-01   -9243055.15
 dtype: float64, 'cash': 2019-01-01    1388.89
 2019-01-02    1388.89
 2019-01-03    1388.89
 2019-01-04    1388.89
 2019-01-05    1388.89
                ...   
 2019-12-27    1388.89
 2019-12-28    1388.89
 2019-12-29    1388.89
 2019-12-30    1388.89
 2019-12-31    1388.89
 Freq: D, Length: 365, dtype: float64, 'payoff': 2020-01-01    10250000.0
 dtype: float64}